In [3]:
#######qcut이용, 0노드 뺴고.?
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# 위도 경도 데이터 로드
latlong_data = pd.read_csv("data/final_num_192.csv")
distance_matrix = pd.read_csv("data/final_distance.csv")

def calculate_total_distance(route, dist_matrix):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += dist_matrix[route[i], route[i+1]]
    return total_distance

def nearest_neighbor(dist_matrix, demands, capacity, cluster_indices):
    num_points = len(cluster_indices)
    visited = np.zeros(num_points, dtype=bool)
    routes = []
    capa = []
    while np.sum(visited) < num_points:
        current_node = np.argmax(~visited)  # 첫 번째 방문하지 않은 노드에서 시작
        current_capacity = 0
        route = [cluster_indices[current_node]]
        visited[current_node] = True
        while True:
            current = route[-1]
            nearest = None
            min_dist = float('inf')
            for i, neighbor in enumerate(cluster_indices):
                if not visited[i] and demands[neighbor] + current_capacity <= capacity and dist_matrix[current, neighbor] < min_dist:
                    nearest = i
                    min_dist = dist_matrix[current, neighbor]
            if nearest is None:
                break
            route.append(cluster_indices[nearest])
            visited[nearest] = True
            current_capacity += demands[cluster_indices[nearest]]
        routes.append(route)
        capa.append(current_capacity)
    return routes, capa

def format_output(routes, vehicle_info, dist_matrix):
    num = 0
    for i, route in enumerate(routes):
        dis = calculate_total_distance(route, dist_matrix)
        num += dis
        vehicle_name, capacity = vehicle_info[i % len(vehicle_info)]
        print(f"Route {i} ({vehicle_name}, Capacity: {capacity}): {dis}")
        print(*route, sep="->")
        print("")
    print("Objective: " + str(num))

def vrp_solver(distance_matrix, latlong_data, vehicle_info):
    demands = np.ones(len(distance_matrix))  # 모든 지점의 수요를 단순화를 위해 1로 가정
    
    # 위도 기준으로 군집화 (qcut 사용)
    latlong_data['cluster_label'] = pd.qcut(latlong_data['x'], q=5, labels=False)
    
    all_routes = []
    all_capa = []
    
    for cluster in range(5):  # 군집 수를 5로 지정
        cluster_indices = np.where(latlong_data['cluster_label'] == cluster)[0]
        cluster_dist_matrix = distance_matrix[np.ix_(cluster_indices, cluster_indices)]
        
        # 각 차량의 용량에 맞는 라우트 계산
        routes, capa = nearest_neighbor(distance_matrix, demands, vehicle_info[list(vehicle_info.keys())[cluster % len(vehicle_info)]], cluster_indices)
        
        all_routes.extend(routes)
        all_capa.extend(capa)
    
    format_output(all_routes, list(vehicle_info.items()), distance_matrix)
    return all_routes, all_capa

# 차량 정보 정의
vehicle_info = {
    '95수9917': 6100,
    '95조7024': 3700,
    '93서5934': 5000,
    '800조2173': 1500,
    '81수0990': 3000
}

# 위도 경도 데이터 및 차량 정보를 사용해 VRP 해결
routes, capa = vrp_solver(distance_matrix.to_numpy(), latlong_data, vehicle_info)

Route 0 (95수9917, Capacity: 6100): 18005
4->169->13->51->64->56->59->151->20->14->170->27->33->57->70->25->54->52->146->142->100->121->6->126->120->117->124->113->114->71->75->34->17->119->171

Route 1 (95조7024, Capacity: 3700): 6512
0->22->101->16->40->11->10->30->44->26->19->42->152->41->74->38->107->150->60->160->157->156->155->153->158->147->148->145->140->8->154->110->141->109

Route 2 (93서5934, Capacity: 5000): 6378
1->18->28->15->48->167->43->168->72->55->29->161->66->165->58->2->163->166->159->24->37->164->32->136->9->36->73->108->162->144->143->21->47->69

Route 3 (800조2173, Capacity: 1500): 8033
3->49->50->39->12->118->123->35->125->116->115->61->135->128->53->45->63->67->7->89->78->83->99->76->81->82->79->98->112->91->131->96->149->65

Route 4 (81수0990, Capacity: 3000): 8521
5->46->106->139->92->77->90->111->62->23->31->95->85->84->93->94->88->97->80->87->86->103->102->132->137->134->130->104->138->68->105->122->127->133->129

Objective: 47449


In [5]:
pd.qcut(latlong_data['x'], q=5)

0      (127.012, 127.014]
1      (127.014, 127.017]
2      (127.014, 127.017]
3       (127.017, 127.02]
4      (126.967, 127.012]
              ...        
167    (127.014, 127.017]
168    (127.014, 127.017]
169    (126.967, 127.012]
170    (126.967, 127.012]
171    (126.967, 127.012]
Name: x, Length: 172, dtype: category
Categories (5, interval[float64, right]): [(126.967, 127.012] < (127.012, 127.014] < (127.014, 127.017] < (127.017, 127.02] < (127.02, 127.024]]